<img src="IMG/PYT_G01_logo.svg" width="100%"/>
<a href="0_Einfuehrung_Inhalt.ipynb" target="_blank">&larr; Einführung/Inhalt</a>

# 9. Threading

Unsere bisherigen Programme liefen immer in einem einzigen Thread (Ausführungsstrang), d.h. alles passierte nacheinander ("seriell"). Dies ist in vielen Fällen unproblematisch, doch es gibt Fälle, in denen man durch "Parallelisierung" (z.B. gleichzeitige Funktionsaufrufe, Berechnungen im Hintergrund etc.) das Programm massiv optimieren kann in Bezug auf die Laufzeit bzw. Effizienz. Für die "Parallelisierung" stehen in Python verschiedene Packages zur Verfügung:

- **asyncio**: asynchrone Ausführung von I/O-Operationen in einem Thread (API-Requests, Datenbank-Zugriff, ...)
- **threading**: parallele Ausführung von Programmteilen in separaten *Threads*
- **multiprocessing**: parallel Ausführung von Programmteilen in separaten *Prozessen*

Wir werden von den oben erwähnten Möglichkeiten ausschliesslich das **Threading** anschauen. Allgemein wird durch einen Thread in der Informatik ein Ausführungsstrang oder eine Ausführungsreihenfolge in der Abarbeitung eines Programmes bezeichnet. In gewissem Sinne kann man Threads auch als eine Erweiterung des Funktions-Konzeptes einer Programmiersprache auffassen. Man kann einen Thread somit wie einen Funktionsaufruf oder Prozeduraufruf sehen, der sozusagen "parallel" zum Hauptstrang ausgeführt wird.

Standardmässig besitzt jeder **Prozess** mindestens einen **Thread**. Ein Prozess kann mehrere Threads starten. Der Vorteil von Threads gegenüber Prozessen besteht darin, dass sich die Threads denselben Speicherbereich für globale Variablen teilen. Verändert ein Thread eine globale Variable, ist der neue Wert auch in dieser Variablen sofort für alle anderen Threads des Prozesses sichtbar. Ein Thread hat aber auch eigene lokale Variablen. Die Verwaltung von Threads ist für das Betriebssystem einfacher, weshalb Threads auch als Leichtgewichtprozesse bezeichnet werden. Der oben beschriebene Vorteil von Threads bringt aber gewisse Risiken mit sich. Vor allem wenn mehrere Threads gleichzeitig auf eine gemeinsame Ressource zugreifen, können schnell mühsam zu debuggende Probleme auftreten. Der Zugriff auf eine solche "gemeinsame" Ressource ist entsprechend zu kontrollieren.

Mehr zum Thema *Threading* finden Sie hier:

- <a href="https://www.python-kurs.eu/threads.php" target="_blank">Threads in Python</a>
- <a href="https://superfastpython.com/threading-in-python/" target="_blank">Python Threading: The Complete Guide</a>

Damit die Thematik fassbarer wird, folgt hier ein Einführungsbeispiel. Hierbei wird ebenfalls aufgezeigt, was passiert, wenn keine Threads eingesetzt werden. Studieren Sie das nachfolgende Beispiel mit den beiden Umsetzungen ohne und mit der Verwendung von Threads sehr genau. Danach sollte die Funktionalität der Threads und deren Implementation und Verwendung klar sein.

## 9.1 Beispiel

Nachfolgend definieren wir eine Funktion `do_work()`, welche nichts anderes tut als eine bestimmte Zeit zu warten. Diese soll eine aufwändige Tätigkeit (z.B. aufwändige Berechnung oder Herunterladen/Analysieren von Daten etc.) repräsentieren, die relativ viel Zeit benötigt und damit das Programm "ausbremsen" kann.

In [7]:
import time

def do_work(n):
    print('Aufruf {} gestartet'.format(n))
    time.sleep(3)  # hier würde die ressourcenintensive Berechnung erfolgen
    print('Aufruf {} beendet'.format(n))

### 9.1.1 Umsetzung ohne Threads ("seriell")

Stellen Sie sich eine Anwendung vor, in der die obige Funktion nun mehrmals ausgeführt werden muss. Dies erfolgt im nachfolgenden Beispiel, in dem die Funktion dreimal aufgerufen wird. In diesem Beispiel wird ohne Parallelisierung (ohne Threads) gearbeitet, d.h. die Aufrufe erfolgen nacheinander (seriell) und damit beträgt die gesamte Laufdauer ca. 3 x 3 s = 9 s. Folgend Abbildung zeigt die Ausführung schematisch.

<center>
<img src="IMG/thread_single.jpg" width=75%/>
</center>

In [8]:
for i in range(3):
    do_work(i+1)

Aufruf 1 gestartet
Aufruf 1 beendet
Aufruf 2 gestartet
Aufruf 2 beendet
Aufruf 3 gestartet
Aufruf 3 beendet


### 9.1.2 Umsetzung mit Threads ("parallel")

Eine massive Zeitersparnis kann erreicht werden, wenn die drei Aufrufe der Funktion nicht nacheinander, sondern parallel erfolgen. Dies können wir mit den bisher gelernten Kontrollstrukturen nicht umsetzen, da die bisherigen Programme immer in nur einem Thread ("seriell") abliefen. Mit **Threads** können wir die Funktionalität in mehrere Threads "aufteilen", welche "parallel" ablaufen. Die folgende Abbildung zeigt die Ausführung schematisch, wobei zu sehen ist, wie die Aufrufe nun "parallel" ablaufen.

<center>
<img src="IMG/thread_multiple.jpg" width=50%/>
</center>

In [11]:
from threading import Thread

for i in range(3):
    t = Thread(target=do_work, args=(i+1,))  # Übergabe der Funktion und der Argumente an Thread t
    t.start()  # Starten des Threads

Aufruf 1 gestartet
Aufruf 2 gestartet
Aufruf 3 gestartet
Aufruf 1 beendet
Aufruf 3 beendet
Aufruf 2 beendet


Wie man erkennt, werden die drei Aufrufe nun parallel gestartet und die Gesamtlaufzeit beträgt damit nur noch ca. 3 s. 

## 9.2 RGB-Beispiel

Das obige Beispiel zeigt die Funktionsweise des Threadings anschaulich, doch nun wollen wir uns noch ein Beispiel anschauen, welches den praktischen Nutzen des Threadings besser illustriert. Dabei geht es darum, einzelne LEDs der RGB-Matrix mit unterschiedlichen Frequenzen blinken zu lassen.

- LED mit Zeilenindex 1 und Spaltenindex 2 soll mit 1 Hz rot blinken
- LED mit Zeilenindex 3 und Spaltenindex 4 soll mit 0.7 Hz grün blinken
- LED mit Zeilenindex 5 und Spaltenindex 6 soll mit 1.3 Hz blau blinken

Überlegen Sie sich kurz, wie Sie diese Aufgabe ohne Threading lösen würden. Es ist durchaus möglich, diese Aufgabe ohne Threading zu lösen, doch Sie werden feststellen, dass Sie nicht einfach mit `time.sleep()` die Blinkfrequenzen der drei LEDs unabhängig voneinander steuern können, da Sie sich gegenseitig blockieren. Speziell mühsam wird das Ganze, wenn die Frequenzen nicht Vielfache voneinander sind. Mithilfe des Threadings ist die Aufgabe deutlich einfacher umzusetzen.

Als erstes erstellen wir eine Funktion `blinken()` mit den Argumenten *position*, *farbe*, *frequenz*, welche die LED in der gewünschten Frequenz und Farbe ein- und ausschaltet. Um Daten zwischen den parallel laufenden Threads austauschen zu können, verwenden wir hier der Einfachheit halber zwei globale Variablen *running* und *matrix*, um das Blinken zu stoppen bzw. die Daten für die RGB-Matrix zu übergeben.

In [7]:
import time

def blinken(position, farbe, frequenz):
    global running, matrix
    while running:
        matrix[position[0]][position[1]] = farbe
        time.sleep((1 / frequenz) / 2)
        matrix[position[0]][position[1]] = (0, 0, 0)
        time.sleep((1 / frequenz) / 2)

Möchte man die Aufgabe ohne Threading umsetzen, kann die obige Funktion nicht einfach gleichzeitig für die drei LEDs aufgerufen werden, da die Aufrufe sich gegenseitig blockieren würden. Mittels Threading können die Aufrufe dagegen in drei separaten Threads ausgeführt werden und somit parallel ablaufen. Mit den nachfolgenden Zeilen werden nun drei Aufrufe der Funktion `blinken()` in separaten Threads gestartet.
```python
for pos, fa, freq in zip(positionen, farben, frequenzen):
    Thread(target=blinken, args=(pos, fa, freq)).start()
```
Diese Threads laufen nun parallel zum Hauptstrang und blockieren diesen bzw. einander nicht. Mit der globalen Variable *running* werden nach ca. 10 s die Threads gestoppt und das Programm endet.
```python
running = False
```
(Nebenbei: Die Aufgabe könnte eleganter mittels OOP realisiert werden, wobei jede blinkende LED ein Objekt einer Blinker-Klasse wäre. Überlegen Sie sich, wie Sie die Aufgabe mittels OOP realisieren würden.)

In [13]:
import time
from threading import Thread
from abbts_blp.rgb import RgbFpga

if __name__ == '__main__':
    positionen = [[1, 2], [3, 4], [5, 6]]
    farben = [(10, 0, 0), (0, 10, 0), (0, 0, 10)]
    frequenzen = [1, 0.4, 2.5]

    rgb = RgbFpga(port='COM5')
    matrix = rgb.rgb_matrix
    rgb.open()
    running = True

    for pos, fa, freq in zip(positionen, farben, frequenzen):
        Thread(target=blinken, args=(pos, fa, freq)).start()

    t_end = time.time() + 10
    while time.time() < t_end:
        rgb.rgb_matrix = matrix
        rgb.write()
    running = False
    rgb.close()


---
<p style='text-align: right; font-size: 70%;'>Grundlagen Python (PYT_G01) / 2024</p>